## Counterfactual explanations: Практика

Среди библиотек, позволяющих реализовать Counterfactual explanations наиболее популярной является [DiCE](https://github.com/interpretml/DiCE/tree/main). Именно с DiCE (Diverse Counterfactual Explanations), мы будем работать, так как она наиболее гибка для использования. Объяснения можно получать для моделей, обученных при помощи `sklearn, keras, tensorflow` и `pytorch`, но только для табличных данных.

Чуть менее известной библиотекой является [CARLA](https://github.com/carla-recourse/CARLA). Если вы решаете задачу классификации рекомендуем к ней присмотреться, так как она более широка *по способам* получения контрфактического объяснения.

In [ ]:
!pip install dice-ml carla -q

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

data = pd.read_csv('https://github.com/SadSabrina/explainable_AI_course/raw/refs/heads/main/data/fetch_california_housing.csv',
                   index_col=0)
data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
0,8.3252,41,6.984127,1.023810,322,2.555556,37.88,-122.23,4.526
1,8.3014,21,6.238137,0.971880,2401,2.109842,37.86,-122.22,3.585
2,7.2574,52,8.288136,1.073446,496,2.802260,37.85,-122.24,3.521
3,5.6431,52,5.817352,1.073059,558,2.547945,37.85,-122.25,3.413
4,3.8462,52,6.281853,1.081081,565,2.181467,37.85,-122.25,3.422


**Контрфактическое объяснение: зачем?**

Контрфактическое объяснение используется в локальном смысле — то есть, когда объясняется прогноз модели для какого-то конкретного объекта из тестовых данных.

**Кейс:** расммотрим задачу прогнозирования цен дома. Вы предоставили модель непосредственному заказчику и необходимо узнать, почему конкретный дом $x[i]$ был оценен в стоимость $y_i$? Какие минимальные изменения необходимо внести, чтобы прогноз принадлежал отрезку $[y_{j-1}, y_j]$?

**Что дает контрфактическое объяснение в случае с регрессией?**

**Ответ:** Как один из возможных вариантов, наиблюдение $x$, такое что $\hat{f}(x)$ принадлежит заданному диапазону $[y_i, y_j]$.

Чтобы не отходить далеко от концепции, разобранной в примере, продолжим работать с нашей прекрасной Калифорнией и её не менее чудесными домами! Заметим, что нас датасет содержит только непрерывные признаки. Для нас это хорошо, поскольку мы можем не углубляться в предобработку данных. Чтобы совсем избежать шага предобработки, будем использовать ансамблиевый алгоритм `RandomForestRegressor`.

**Quiz 1:** Какие ещё методы объяснения из уже изученных мы могли бы применить к данному алгоритму?

In [ ]:
#Подготовим данные для обучения

X = data.drop('target', axis=1)
y = data['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from IPython.display import display

forest = RandomForestRegressor(random_state=42)
params = {'max_depth': [6, 10, 14]}

gs = GridSearchCV(forest, params, cv=3, scoring='neg_mean_squared_error')
gs.fit(X_train, y_train)

estimator = gs.best_estimator_

print('Cv score:')
display(pd.DataFrame(gs.cv_results_))
print('Test MSE:', mean_squared_error(estimator.predict(X_test), y_test))

Cv score:


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,3.544557,0.399885,0.034979,0.001471,6,{'max_depth': 6},-0.406816,-0.385642,-0.405898,-0.399452,0.009773,3
1,5.735554,0.504664,0.058271,0.001342,10,{'max_depth': 10},-0.308077,-0.291431,-0.300630,-0.300046,0.006808,2
2,6.788484,0.493756,0.097019,0.003415,14,{'max_depth': 14},-0.279224,-0.271016,-0.277133,-0.275791,0.003483,1


Test MSE: 0.2612408079794064


##**DiCE**



Чтобы найти ближайших котрфактический объект, мы должны определить:
- по какому критерию искать (контрфактность прогноза)
- где искать
- как искать
- для кого искать

Отвечать на эти вопросы, используя DiCE (как для классификации, так и для регрессии), мы будем определять объекты двух класов: Data и Model. В случае, если вы хотите сохранить приватность данных, то DiCE также предоставляет эту возможность (её мы также разберем чуть ниже).

Для использования `Data` и `Model` необходимо:

- сохранить названия непрерывных и категориальных признаки в списки `continuous_features` и `discrete_features` соответственно
- обучить модель

Свой запрос к контрфактическому объяснению мы формулируем как:

`Во всём наборе данных (его мы тебе дадим) найди ближайшие контрфакты для выбранного экземпляра, такой что прогноз $\hat{f}(x)$ для него $\in [y_i, y_j]$`

1. **Где искать** — указываем в объекте класса `Data`. Это может быть любое множество данных, структура которого такая же, как и у данных, на которых обучена модель.
2. **Как искать** — здесь DiCE предлагает три метода поиска `kdtree`, `genetic` и `random`. От выбранного метода в том числе зависят найденные контрфакты. То есть такого, что вы нашли 3 контрфакта и они устойчивы от метода поиска к методу поиска **не будет**. Устойчивые от итерации к итерации контрфакты дает `kdtree`, поэтому мы рекомендуем использовать его.
3. **Для кого искать** - любой объект из тестового набора данных.

**Где искать**

Искать контрфактический пример можно как в тренировочном, так и в тестовом наборах данных.

**Тренировочный набор данных** \

Цель: деббагинг модели в процессе обучения. \

**Преимущества:**
- Можно использовать все доступные данные для анализа, так как обучабщая выборка — самое большое доступное нам множество данных.
- Можно лучше понять, как модель обучается и какие признаки наиболее влияют на предсказания. Это в некоторых случаях помогает убрать нежелательные объекты или скорректировать распределения некоторых признаков.

**Недостатки:**
- Контрфактические примеры могут не отражать реальность, если модель переобучена на тренировочных данных.
- Найденные примеры могут быть не релевантными для новых, ранее невидимых данных.

**Тестовый набор данных**

Цель: Оценка и интерпретация модели на новых, невидимых данных.

**Преимущества:**
- Позволяет проверить, как модель будет вести себя нв реальных сценариях.

**Недостатки:**
- Меньшее количество данных для анализа по сравнению с тренировочным набором.

**Общие рекомендации:**

- Для исследовательских целей и интерпретации модели во время её разработки используйте тренировочный набор данных. Это поможет лучше понять внутреннюю работу модели и её реакции на изменения признаков.
- Для оценки модели и её поведения на новых данных: используйте тестовый набор данных. Это поможет проверить, как модель будет работать в реальных условиях и насколько устойчивы её предсказания к изменениям в данных.

Мы рассмотрим сценарий поиска контрфактического объекта в тестовых данных `X_test`. Хотя для определения класса `Data` необходимо, чтобы используемый набор полностью удовлетворял структуре данных, на которых обучена модель, нам не обязательно включать в `test` значения целевой веременной. Достаточно поставить на них "заглушку" любым значением.

In [ ]:
sample = X_test.sample(n=20, random_state=42)
data_to_search = sample.copy()

X_train_dice = X_train.copy()
X_train_dice['target'] = y_train

In [ ]:
import dice_ml
from dice_ml import Dice

continuous_features= list(X.columns)

d_housing = dice_ml.Data(dataframe=X_train_dice, outcome_name='target', continuous_features=continuous_features)

m_housing = dice_ml.Model(model=estimator, backend="sklearn", model_type='regressor')

Реализация поиска контрфактического объяснения проводится при помощи объекта класса `Dice`. На выбор в DiCE включены *3 метода поиска*: `genetic`, `kdtree`, `random`.

**На что влияет метод поиска:**
- на **скорость** поиска — (I) `random`, (II) `genetic`, (III) `kdtree`, время меняется в заивисимости от количества наблюдений, для которых ищутся контрфактические наблюдения (не обязательно искать только для одного экземпляра данных
- на **устойчивость** поиска — найденный контрфактический объект не обязан быть единственным или лучшим из-за чего методы поиска могут выдавать разные объекты от итерации к итерации. Наиболее устойчив (проверено на малых данных) `kdtree`. Мы рекомендуем использовать его.

In [ ]:
exp_genetic_housing = Dice(d_housing, m_housing, method="kdtree") #инициализируем объект, при помощи которого будем искать контрфактическое объяснение

query_instance_housing = data_to_search[:1]

print('Model prediction:', """Ваш код здесь""")

In [ ]:
query_instance_housing

**Зададимся вопоросом:** как минимально поменять объект, чтобы прогноз модели увеличился в 2 раза?

**Quiz 2:** Задайте отрезок для поиска под данную задачу вида: $[y\_twice, y\_twice + 0.5]$. В ответ на степик запишите правую границу (округлите до сотых).

In [ ]:
genetic_housing_twice = exp_genetic_housing.generate_counterfactuals(query_instance_housing,
                                                               total_CFs=3,
                                                               desired_range=#Ваш код здесь)
genetic_housing_twice.visualize_as_dataframe(show_only_changes=True)

100%|██████████| 1/1 [00:00<00:00,  1.93it/s]

Query instance (original outcome : 1.8664358854293823)


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
0,4.0,35,4.938095,0.985714,552,2.628572,37.709999,-122.120003,1.866436



Diverse Counterfactual set (new outcome: [3.73287176, 4.23])


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
15803,4.05,-,4.1,1.1,-,1.5,-,-,4.201006889343262
3744,3.806,-,5.2,1.1,501.0,1.9,34.16,-118.4,3.818920135498047
3782,5.158,37.0,5.4,1.1,-,2.1,34.16,-118.4,4.1716718673706055


**Quiz 3:** Задайте отрезок для поиска:  $[4, 5]$ \
В ответ введите значение переменной Population для найденных объектов

In [ ]:
genetic_housing_twice_up = exp_genetic_housing.generate_counterfactuals(query_instance_housing,
                                                               total_CFs=3,
                                                               desired_range=#Ваш код здесь)
genetic_housing_twice_up.visualize_as_dataframe(show_only_changes=True)

100%|██████████| 1/1 [00:00<00:00,  1.86it/s]

Query instance (original outcome : 1.8664358854293823)


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
0,4.0,35,4.938095,0.985714,552,2.628572,37.709999,-122.120003,1.866436



Diverse Counterfactual set (new outcome: [4, 5])


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
18279,7.707,-,7.1,1.1,-,2.9,37.35,-,4.9799933433532715
18353,10.736,37.0,7.6,1.0,-,2.9,37.37,-,4.999934673309326
8576,5.96,43.0,4.7,1.1,-,1.9,33.9,-118.4,4.984320640563965


**Какой из найденных ближайший? Евклидова метрика**

Расстояние между объектами $x, y$ по определению — это совсем не $\sqrt{\sum_i^n (x_i - y_i)^2}$.

В математике расстояние является *метрикой* и объявляется аксиоматически. Не пугайтесь, вам не обязательно знать аксиомы. Из этого факта вам необходимо понимать, что расстояние — это один из способов оценить удалённость объектов друг от друга.

Расстояние, которое вводят в школе ($p(x, y) = \sqrt{\sum_i^n (x_i - y_i)^2}$) — это лишь один из примеров. Но им можно и нужно пользоваться! Оно называется Евклидовым расстоянием. Если ваши табличные данные отражают специфическую структуру, например, текст (допустим, в виде Tf-idf-кодирования), то можем быть продуктивнее рассмотреть другой вид расстояния.  



In [ ]:
#Сохраним найденные для 2x интервала объекты в виде списка
cf_objects = genetic_housing_twice.cf_examples_list[0].final_cfs_df.values

Вычислими Евклидово расстояние между объясняемым и найденными примерами. Расстояние будем считать по все координатам, кроме целевой.

In [ ]:
def euclidean_distance(x, y):

  distance = np.sqrt(np.sum((x-y)**2))
  return distance


def cheb_distance(x, y):

  distance = np.max(np.abs((x-y)))
  return distance

In [ ]:
query_instance_housing_as_vector = query_instance_housing.values

**Quiz 4:** Посчитайте Евклидово расстояние между найденными контрфактическими объектами и объясняемым примером. Чему равно наименьшее? Ответ округлите до сотых.

In [ ]:
for i in cf_objects:
  print('Euclidean dist:', #Ваш код здесь)


Если же нам просто важно, чтобы изменения были наименьшими в каждой координате, то можно вычислить расстояние Чебышева по формуле $p(x, y) = max(|x_i-y_i|)$

**Quiz 5:** Рассчитайте расстояние Чебышева для тех же объектов. Поменялся ли результат? Выберите ответы, которые соответствуют выводам из получившихся значений.

Проведите собственный анализ для объекта номер 7, не меняя способ поиска объяснений. Контрафактические объекты ищите в отрезке $[model\_prediction*2, model\_prediction*2 + 0.5]$

In [ ]:
query_instance_housing_two = #Ваш код здесь
print('Model prediction', estimator.predict(query_instance_housing_two))

genetic_housing_seven = exp_genetic_housing.generate_counterfactuals(query_instance_housing_two,
                                                               total_CFs=3,
                                                               desired_range=#Ваш код здесь)

genetic_housing_seven.visualize_as_dataframe(show_only_changes=True)

**Quiz 6:** Чему равен наибольший таргет среди нафденных 3х объектов? Ответ округлите до сотых.

**Работа в DiCE без доступа к тренировочным данным**

На самом деле, мы с вами уже почти разобрали этот шаг. DiCE является методом, который больше зависит от модели и того, как она была обучена, соответственно показывать тренировочные данные нет необходимости. Но некоторые статистики данных знать всё-таки нужно, поскольку методу нужно понимать, где искать.  Иначе, к сожалению, результатов метода не получить.

Итак, чтобы работать с анонимизированными данными достаточно перечислить все признаки и их возможные значения в атрибуте `features`. Соберем простой пример — будем прогнозировать класс уже знакомых вам Ирисов, чтобы не переобучать новую модель долго.

In [ ]:
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

X, y = load_iris(return_X_y=True, as_frame=True)

X_tr, X_te, y_tr, y_te = train_test_split(X, y, random_state=42)

model = DecisionTreeClassifier(max_depth=3)
model.fit(X_tr, y_tr)

print('Model accuracy:', accuracy_score(model.predict(X_te), y_te))

Model accuracy: 1.0


Рассмотрим список признаков и диапазоны их значений.

In [ ]:
X_tr.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
count,112.000000,112.000000,112.000000,112.000000
mean,5.830357,3.040179,3.807143,1.214286
std,0.819123,0.437120,1.735310,0.747953
min,4.300000,2.000000,1.100000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.300000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.700000,4.200000,6.700000,2.500000


Собираем данные, указывая границы числовых признаков. Для категориальных необходимо было бы перечислить категории. Порядок признаков должен быть тот же, что и при обучении модели.

In [ ]:

d_without_access = dice_ml.Data(features={'sepal length (cm)': [4.3, 7.7],
                                          'sepal width (cm)': [2, 4.2],
                                          'petal length (cm)': [1.1, 6.7],
                                          'petal width (cm)' : [0.1, 2.5]},
                 outcome_name='target')

Класс model объявляем практически также. Вместо модели, обученной непосредственно в течение ноутбука, можно также добавить путь к сохраненной предобученной модели в параметр 'model_path'.

In [ ]:
backend = 'sklearn'
m = dice_ml.Model(model=model, backend=backend, model_type='classifier')

query_instance = pd.DataFrame({'sepal length (cm)': 12,
                               'sepal width (cm)': 3,
                               'petal length (cm)': 1,
                               'petal width (cm)': 0.7},
                               index=[0])

Объявляем "поиск" и получаем результаты. Однако история без доступа к данным имеет значительный минус — невозможно искать наиболее усточивым kdtree.

In [ ]:
exp = dice_ml.Dice(d_without_access, m, method="genetic")

dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=3, desired_class=2,  initialization="random"
                                       )

dice_exp.visualize_as_dataframe(show_only_changes=True)

100%|██████████| 1/1 [00:00<00:00,  5.37it/s]


Query instance (original outcome : 0)


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,12,3,1,0.7,0



Diverse Counterfactual set without sparsity correction since only metadata about each  feature is available (new outcome: 2


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.0,-,-,2.0,2.0
0,4.0,-,-,2.0,2.0
0,4.0,-,-,2.0,2.0


**Quiz 7:** Получилось ли получить устойчивые контрфактические объяснения?

## **Выводы**
В этом разделе мы познакомились с объяснением моделей при помощи геренации контрфактуальных объяснений при помощи DiCE. Мы выяснили, что:
- необходимо четко формулировать сэмпл данных, на основе которого будут генерироваться контрфактические значения
- метод не всегда приносит устойчивые объяснения
- для уточнения близости могут быть использованы метрики расстояний
- метод не требует прямого доступа к тренировочным данным.